In [1]:
import sys
import glob
import pickle
import numpy as np
import re
from datetime import datetime

import functions as fxn #make sure you have functions.py file in the same dir

/anaconda3/envs/pythondata/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import re

In [3]:
import os

In [4]:
from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Bidirectional, Dense, Dropout, LSTM, Activation
from keras.callbacks import ModelCheckpoint, TensorBoard, History, Callback
from keras.utils import np_utils

In [5]:
import glob

In [6]:
from music21 import converter, note, chord, stream, instrument

In [7]:
def full_execution(midi_files, output_tag, total_epochs, batch_size, sequence_length, temperature, offset_adj):
    # epoch_stops = 1
    # epoch_count = 0
    weight_file = None

    note_file = fxn.convert_midis_to_notes(midi_files, output_tag)

    epochs = total_epochs
    with open(note_file, 'rb') as filepath:
        notes = pickle.load(filepath)
    network_input, network_output, n_patterns, n_vocab, pitchnames = fxn.prepare_sequences(notes, sequence_length)
    network_input_r, network_output_r = fxn.reshape_for_training(network_input, network_output,sequence_length)

    # while epoch_count <= total_epochs:
    #     epochs = epoch_stops
    #
    model = fxn.create_network(network_input_r, n_vocab, weight_file)
    model, weight_file, history = fxn.train_model(model, network_input_r, network_output_r, epochs, batch_size, output_tag, sequence_length)
    normalized_input = fxn.reshape_for_creation(network_input, n_patterns, sequence_length, n_vocab)
    model = fxn.create_network(normalized_input, n_vocab, weight_file)
    prediction_output= fxn.generate_notes(model, network_input, pitchnames,sequence_length, notes_generated, n_vocab, temperature)
    output_notes = fxn.create_midi(prediction_output, output_tag, sequence_length, offset_adj)
        # epoch_count += epoch_stops
    return output_notes, history, weight_file

In [8]:
timestamp = re.sub(r'[-: ]','',str(datetime.now()).split('.')[0])[:-2]
total_epochs = 70

batch_size = 128 # 128 for local;
sequence_length = 200 # the LSTM RNN will consider this many notes

notes_generated = 500 # Change the # of notes generated if you want
temperature = 1.0
offset_adj = 0.5

# Put path to midi folder; use [:max] for number of songs to be used
midi_files = '/Users/lwang/FinalProjectLu/Data/Bollywood/*.mid'

output_name = midi_files.split('/')[-2]


In [9]:
#Put the path for output files
test = '{}_{}'.format(timestamp, output_name) 
output_tag = os.path.join('Data', 'Output', test) 
terminal_output = output_tag + 'terminal.log'

In [10]:
files = glob.glob('/Users/lwang/FinalProjectLu/Data/Bollywood/*mid')

In [11]:
for file in files:
    print(file)

/Users/lwang/FinalProjectLu/Data/Bollywood/kareeb_chori_chori_jab_nazare_mili.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/mein_khiladi_tu_anadi_chura_ke_dil_mera.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/jewel_theif_ye_dil_na_hota_bechara.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/quayamat_se_quayamat_tak_pehla_nasha.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/zubeida_dheeme_dheeme.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/khal_nayak_choli_ke_peeche_kya_hai.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/roja_bharat_humko_jaan_se_pyara_hai.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/punjabi_mc_mundian_to_bach_ke.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/sirf_tum_dilbur_dilbur.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/maine_pyar_kiya_mere_sawalon_ka_jawab_do.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/rehna_hai_tere_mein_ae_bolo_bolo.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/aashiqui_ab_teray_bin_jee_lenge_hum.mid
/Users/lwang/FinalProjectLu/Data/

In [12]:
sys.stdout = fxn.Logger(terminal_output)
print("Terminal output being saved at {}".format(terminal_output))
full_execution(midi_files, output_tag, total_epochs, batch_size, sequence_length, temperature, offset_adj)
print("Run Complete. Terminal log saved at {}".format(terminal_output))

Terminal output being saved at Data/Output/201901091745_Bollywoodterminal.log

**Loading Midi files**
/Users/lwang/FinalProjectLu/Data/Bollywood/kareeb_chori_chori_jab_nazare_mili.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/mein_khiladi_tu_anadi_chura_ke_dil_mera.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/jewel_theif_ye_dil_na_hota_bechara.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/quayamat_se_quayamat_tak_pehla_nasha.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/zubeida_dheeme_dheeme.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/khal_nayak_choli_ke_peeche_kya_hai.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/roja_bharat_humko_jaan_se_pyara_hai.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/punjabi_mc_mundian_to_bach_ke.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/sirf_tum_dilbur_dilbur.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/maine_pyar_kiya_mere_sawalon_ka_jawab_do.mid
/Users/lwang/FinalProjectLu/Data/Bollywood/rehna_hai_tere_mein_ae_bolo_bolo.mid
/Users/lwang/F

Network input and output created with (pre-transform) lengths 58839 and 58839
Lengths. N Vocab: 570 N Patterns: 58839 Pitchnames: 570

**Reshaping for training**
Reshaping network input to (notes - sequence length, sequence length) (58839, 200, 1)
Reshaping network output to (notes - sequence length, unique notes) (58839, 570)

**LSTM model initializing**
Input nodes: 512 Dropout: 0.5
Input shape (timesteps, data_dim): (200,1)
expected str, bytes or os.PathLike object, not NoneType
No weights loaded
Fitting Model. 
Network Input Shape: (58839, 200, 1) Network Output Shape: (58839, 570)
Epochs: 70 Batch Size: 128
Train on 47071 samples, validate on 11768 samples
Epoch 1/70
47071/47071 [==============================] - 1780s 38ms/step - loss: 5.6830 - val_loss: 4.8566

Epoch 00001: saving model to Data/Output/201901091745_Bollywoodweight-01-5.6830-4.8566.hdf5
Epoch 2/70
47071/47071 [==============================] - 1751s 37ms/step - loss: 5.3795 - val_loss: 4.6085

Epoch 00002: saving 

47071/47071 [==============================] - 1793s 38ms/step - loss: 4.9463 - val_loss: 4.6802

Epoch 00038: saving model to Data/Output/201901091745_Bollywoodweight-38-4.9463-4.6802.hdf5
Epoch 39/70
47071/47071 [==============================] - 1892s 40ms/step - loss: 4.9595 - val_loss: 4.6268

Epoch 00039: saving model to Data/Output/201901091745_Bollywoodweight-39-4.9595-4.6268.hdf5
Epoch 40/70
47071/47071 [==============================] - 1878s 40ms/step - loss: 4.9453 - val_loss: 4.6206

Epoch 00040: saving model to Data/Output/201901091745_Bollywoodweight-40-4.9453-4.6206.hdf5
Epoch 41/70
47071/47071 [==============================] - 1844s 39ms/step - loss: 4.9413 - val_loss: 4.5589

Epoch 00041: saving model to Data/Output/201901091745_Bollywoodweight-41-4.9413-4.5589.hdf5
Epoch 42/70
47071/47071 [==============================] - 1773s 38ms/step - loss: 4.9556 - val_loss: 4.6888

Epoch 00042: saving model to Data/Output/201901091745_Bollywoodweight-42-4.9556-4.6888.hdf5
Ep

Network Input of length 58839 is reshaped to normalized input of (58839, 200, 1)

**LSTM model initializing**
Input nodes: 512 Dropout: 0.5
Input shape (timesteps, data_dim): (200,1)
Weights file loaded

**Generating notes**
Length Pitchnames: 570
Integer to note conversion at length: 570
Network input length: 58839
Pattern ends with length 200 and type <class 'list'>
Generated Note Length: 500
First 100: ['9', 'E2', '3.7', '3.7', '6.9', '9.0.1.3.4', 'G6', '10.1', '1.4.6', '6.9.1', 'F5', '2.4.6', '3.7', '4.8.10.11', 'E2', '11.3.4.6', '3.7', '6.8.1', '3.7', '9', '0.1.4.6.9', '6.10.1', '6.7.9.10.1', '6.9.11', '5.9', 'F#3', 'B-2', '1.3.6', 'B-1', '7.11', 'B2', '11.1', 'E4', '5.9.0', '3.7', '0.3', '10.1', 'B-5', '2.4.6', '10', '5.9.0', '10.1.4', '11.2', '11.3.4.6', '3.7', '6.7.11.2', 'D2', '3.7', 'D7', '11.3.4.6', 'C3', 'C3', '11.3.4.6', '2.4.6', '6.7', '3.7', '3.7', '6.9.1', '3.7.10', 'B-3', '4.6.9.0', 'E-3', '4', 'B-2', '5.9.0', 'E2', 'F#2', '6.8.1', '2.4.6', '4.6.8', '0.3.7', 'C#1', 'E3